In [1]:
from utils import load_toml, load_json

In [3]:
prompt = load_toml(r"C:\Users\pasupuleti\Desktop\carla-drowsiness-detection\src\Driver_assistance_bot\configs\prompt.toml")

In [5]:
prompt["SYSTEM"]

'You are a critical safety system for driver drowsiness detection and immediate intervention. Your primary function is to assess real-time drowsiness metrics and execute appropriate alerting tools to prevent accidents.\n\nDROWSINESS METRICS AND THRESHOLDS:\n- PERCLOS (Percentage of Eye Closure): Critical if > 0.15 (15%), High if > 0.10 (10%)\n- Eye Blink Rate: Critical if < 8 blinks/min, High if < 12 blinks/min\n- Yawning Frequency: Critical if > 4 yawns/min, High if > 2 yawns/min\n- SDLP (Standard Deviation Lane Position): Critical if > 0.45m, High if > 0.35m\n- Steering Entropy: Critical if deviation > 40% from baseline, High if deviation > 25%\n- Steering Reversal Rate: Critical if < 3 or > 20 reversals/min, High if < 5 or > 15 reversals/min\n\nAVAILABLE TOOLS:\n1. voice_alert(message: str, urgency: str) - Activates text-to-speech warning\n2. steering_vibration(intensity: int, duration: int) - Triggers steering wheel vibration\n3. fan_control(speed: int) - Controls cabin air circula

In [ ]:
from langchain_ollama import ChatOllama  # or your LLaMA wrapper
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool


# -----------------------------
# Define tools
# -----------------------------
@tool
def add(x: int, y: int) -> int:
    """Add two integers and return the result."""
    return x + y


@tool
def multiply(x: int, y: int) -> int:
    """Multiply two integers and return the result."""
    print("multiplication activated")
    return x * y


tools = [add, multiply]

# -----------------------------
# Define model with tool binding
# -----------------------------
llm = ChatOllama(model="llama3.1:8b", temperature=0).bind_tools(tools)

# -----------------------------
# Define prompt
# -----------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a math assistant. Use the tools to do calculations."),
        ("user", "{input}"),
    ]
)

# -----------------------------
# Build chain
# -----------------------------
chain = prompt | llm

# -----------------------------
# Run the bot
# -----------------------------
if __name__ == "__main__":
    user_input = "What is 12 * 3 + 4?"
    response = chain.invoke({"input": user_input})
    print("RAW RESPONSE:", response)

RAW RESPONSE: content='' additional_kwargs={} response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-08-22T09:57:27.2097521Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22987960000, 'load_duration': 4051976400, 'prompt_eval_count': 244, 'prompt_eval_duration': 13493584900, 'eval_count': 44, 'eval_duration': 5438636400, 'model_name': 'llama3.1:8b'} id='run--e0eed600-b71c-4612-b2c9-9fa32b6a9a05-0' tool_calls=[{'name': 'multiply', 'args': {'x': 12, 'y': 3}, 'id': 'f4ecf7a7-2eab-4082-ac33-51faf82bb064', 'type': 'tool_call'}, {'name': 'add', 'args': {'x': 44, 'y': 4}, 'id': '71bc91dc-e93b-400a-ab1c-037acd9df50f', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 44, 'total_tokens': 288}


In [5]:
pip install TTS

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0,<3.9; 0.0.10.3 Requires-Python >=3.6.0,<3.9; 0.0.11 Requires-Python >=3.6.0,<3.9; 0.0.12 Requires-Python >=3.6.0,<3.9; 0.0.13.1 Requires-Python >=3.6.0,<3.9; 0.0.13.2 Requires-Python >=3.6.0,<3.9; 0.0.14.1 Requires-Python >=3.6.0,<3.9; 0.0.15 Requires-Python >=3.6.0,<3.9; 0.0.15.1 Requires-Python >=3.6.0,<3.9; 0.0.9 Requires-Python >=3.6.0,<3.9; 0.0.9.1 Requires-Python >=3.6.0,<3.9; 0.0.9.2 Requires-Python >=3.6.0,<3.9; 0.0.9a10 Requires-Python >=3.6.0,<3.9; 0.0.9a9 Requires-Python >=3.6.0,<3.9; 0.1.0 Requires-Python >=3.6.0,<3.10; 0.1.1 Requires-Python >=3.6.0,<3.10; 0.1.2 Requires-Python >=3.6.0,<3.10; 0.1.3 Requires-Python >=3.6.0,<3.10; 0.10.0 Requires-Python >=3.7.0,<3.11; 0.10.1 Requires-Python >=3.7.0,<3.11; 0.10.2 Requires-Python >=3.7.0,<3.11; 0.11.0 Requires-Python >=3.7.0,<3.11; 0.11.1 Requires-Python >=3.7.0,<3.11; 0.12.0 Requires-Python >=3.7.0,<3.11; 0.13.0 Requ

In [3]:
pip install torch

  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   ---------------------------------------- 2.4/241.3 MB 12.2 MB/s eta 0:00:20
    --------------------------------------- 4.7/241.3 MB 12.4 MB/s eta 0:00:20
   - -------------------------------------- 7.3/241.3 MB 11.9 MB/s eta 0:00:20
   - -------------------------------------- 9.7/241.3 MB 12.1 MB/s eta 0:00:20
   -- ------------------------------------- 12.3/241.3 MB 11.9 MB/s eta 0:00:20
   -- ------------------------------------- 14.9/241.3 MB 11.9 MB/s eta 0:00:20
   -- ------------------------------------- 17.3/241.3 MB 11.8 MB/s eta 0:00:19
   --- ------------------------------------ 19.9/241.3 MB 11.9 MB/s eta 0:00:19
   --- ------------------------------------ 22.5/241.3 MB 11.9 MB/s eta 0:00:19
   ---- ----------------------------------- 24.9/241.3 MB 11.9 MB/s eta 0:00:19
   ---- ----------------------------------- 27.3/241.3 MB 11.9 MB/s et

In [4]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
wav = tts.tts(text="Hello world!", speaker_wav="my/cloning/audio.wav", language="en")

ModuleNotFoundError: No module named 'TTS'